Note: Multiplier has been taken as it is. For averages and ratios it doesn't affect the analysis. To use absolute population values, use multiplier/100 instead

# Importing libraries

In [41]:
import pandas as pd
import numpy as np
import pathlib
import pickle
from functools import reduce

In [42]:
pd.set_option('display.max_columns', None)

# Importing datasets

In [43]:
# Checking paths
pathlib.Path("../../HCES 2023-24/Python implementation/Data extraction/Population based MPCE/MPCE 2023_24 2Apr25.dta").resolve().exists()

True

In [44]:
# Reading HH-wise MPCE
path_mpce = "../../HCES 2023-24/Python implementation/Data extraction/Population based MPCE/MPCE 2023_24 2Apr25.dta"
df_mpce = pd.read_stata(path_mpce)
df_mpce = df_mpce.set_index('hhid')
df_mpce = df_mpce[['hh_size', 'sector', 'mpce', 'decile']]
display(df_mpce.head())

## Level 01 file
path_lvl01 = "../../HCES 2023-24/Dta raw files//level_01.dta"
df_lvl01 = pd.read_stata(path_lvl01)
df_lvl01 = df_lvl01.set_index('hhid')
df_lvl01 = df_lvl01[['state']]
display(df_lvl01.head())

## Level 03 file
path_lvl03 = "../../HCES 2023-24/Dta raw files//level_03.dta"
df_lvl03 = pd.read_stata(path_lvl03)
df_lvl03 = df_lvl03.set_index('hhid')
df_lvl03 = df_lvl03[['source_cooking', 'multiplier']]
display(df_lvl03.head())

## Level 07 file
path_lvl07 = "../../HCES 2023-24/Dta raw files/level_07.dta"
df_lvl07 = pd.read_stata(path_lvl07)
df_lvl07 = df_lvl07.set_index('hhid')
df_lvl07 = df_lvl07[['received_subsidy_lpg', 'num_subsidized_lpg']]
display(df_lvl07.head())

## Level 08 file
path_lvl08 = "../../HCES 2023-24/Dta raw files/level_08.dta"
df_lvl08 = pd.read_stata(path_lvl08)
df_lvl08 = df_lvl08.loc[df_lvl08['item_code'].isin([331,338])]

,hh_size,sector,mpce,decile
hhid,,,,
22300101,4.0,2,9407.24,9
22300201,4.0,2,25398.56,10
22300202,2.0,2,11762.50,10
22300203,2.0,2,11579.24,10
22300204,2.0,2,15350.10,10


,state
hhid,
46667201,1
46667301,1
46667302,1
46667303,1
46667304,1


,source_cooking,multiplier
hhid,,
22300101,2,57436
22300201,2,27497
22300202,2,27497
22300203,2,27497
22300204,2,27497


,received_subsidy_lpg,num_subsidized_lpg
hhid,,
22300101,1,2.0
22300201,1,2.0
22300202,1,1.0
22300203,2,NaN
22300204,1,1.0


# Data wrangling

In [45]:
df_lvl08.head()

,index,hhid,questionnaire_num,level,item_code,cons_home_qty,cons_home_value,cons_total_qty,cons_total_value,source,multiplier
5,5,22300101,C,8,338,NaN,NaN,14.2,860.0,1.0,57436
9,9,22300201,C,8,338,NaN,NaN,14.2,860.0,1.0,27497
16,16,22300202,C,8,338,NaN,NaN,9.5,640.0,1.0,27497
23,23,22300203,C,8,338,NaN,NaN,9.0,720.0,1.0,27497
26,26,22300204,C,8,338,NaN,NaN,9.4,570.0,1.0,27497


In [46]:
# Reshaping level 08 file for firewood and LPG

## Pivoting
df_lvl08 = df_lvl08.pivot_table(index = 'hhid', columns = 'item_code', values = ['cons_total_qty','cons_total_value', 'cons_home_qty', 'cons_home_value' ])

## Converting columns to single level index
df_lvl08.columns = ["_".join(map(str, x)) for x in df_lvl08.columns]

## Changing column names
df_lvl08 = df_lvl08.rename(columns = {'cons_total_qty_331': 'firewood_qty', 'cons_total_qty_338': 'lpg_qty',
                               'cons_total_value_331': 'firewood_value', 'cons_total_value_338': 'lpg_value',
                               'cons_home_qty_331': 'firewood_home_qty', 'cons_home_value_331': 'firewood_home_value',
                               'cons_home_qty_338': 'lpg_home_qty', 'cons_home_value_338': 'lpg_home_value'})
df_lvl08.head()

# Adding market quantity and value
df_lvl08 = df_lvl08.fillna(0)
df_lvl08[['firewood_mkt_qty', 'firewood_mkt_value']] = df_lvl08[['firewood_qty', 'firewood_value' ]].values - df_lvl08[['firewood_home_qty', 'firewood_home_value' ]].values
df_lvl08[['lpg_mkt_qty', 'lpg_mkt_value']] = df_lvl08[['lpg_qty', 'lpg_value' ]].values - df_lvl08[['lpg_home_qty', 'lpg_home_value' ]].values

In [47]:
df_lvl08.head()

,firewood_home_qty,lpg_home_qty,firewood_home_value,lpg_home_value,firewood_qty,lpg_qty,firewood_value,lpg_value,firewood_mkt_qty,firewood_mkt_value,lpg_mkt_qty,lpg_mkt_value
hhid,,,,,,,,,,,,
22300101,0.0,0.0,0.0,0.0,0.0,14.2,0.0,860.0,0.0,0.0,14.2,860.0
22300201,0.0,0.0,0.0,0.0,0.0,14.2,0.0,860.0,0.0,0.0,14.2,860.0
22300202,0.0,0.0,0.0,0.0,0.0,9.5,0.0,640.0,0.0,0.0,9.5,640.0
22300203,0.0,0.0,0.0,0.0,0.0,9.0,0.0,720.0,0.0,0.0,9.0,720.0
22300204,0.0,0.0,0.0,0.0,0.0,9.4,0.0,570.0,0.0,0.0,9.4,570.0


In [48]:
# Merging all files

## Checking shapes
print(f"Shape of MPCE file: {df_mpce.shape}")
print(f"Shape of level 01 file: {df_lvl01.shape}")
print(f"Shape of level 03 file: {df_lvl03.shape}")
print(f"Shape of level 07 file: {df_lvl07.shape}")
print(f"Shape of level 08 file: {df_lvl08.shape}")

## Defining merge function
def merge_df (left_df, right_df):
    df = pd.merge(left_df, right_df, how = 'outer', left_index=True, right_index=True, indicator=True)
    print(f"\nSummary from merging dataframes")
    display(df['_merge'].value_counts())
    df = df.drop(labels = '_merge', axis = 1)
    return df.copy()

## Merging
df = reduce(merge_df, [df_mpce, df_lvl01, df_lvl03, df_lvl07, df_lvl08])

Shape of MPCE file: (261953, 4)
Shape of level 01 file: (261953, 1)
Shape of level 03 file: (261953, 2)
Shape of level 07 file: (261953, 2)
Shape of level 08 file: (251119, 12)

Summary from merging dataframes


_merge
both          261953
left_only          0
right_only         0
Name: count, dtype: int64


Summary from merging dataframes


_merge
both          261953
left_only          0
right_only         0
Name: count, dtype: int64


Summary from merging dataframes


_merge
both          261953
left_only          0
right_only         0
Name: count, dtype: int64


Summary from merging dataframes


_merge
both          251119
left_only      10834
right_only         0
Name: count, dtype: int64

In [49]:
# Renaming column values
df['sector'] = df['sector'].replace({1: 'rural', 2: 'urban'})
df['state'] = df['state'].replace({1: 'Jammu & Kashmir', 2: 'Himachal Pradesh', 3: 'Punjab', 4: 'Chandigarh(U.T.)', 5: 'Uttrakhand', 6: 'Haryana', 7: 'Delhi', 8: 'Rajasthan', 9: 'Uttar Pradesh', 10: 'Bihar', 11: 'Sikkim', 12: 'Arunachal Pradesh', 13: 'Nagaland', 14: 'Manipur', 15: 'Mizoram', 16: 'Tripura', 17: 'Meghalaya', 18: 'Assam', 19: 'West Bengal', 20: 'Jharkhand', 21: 'Odisha', 22: 'Chattisgarh', 23: 'Madhya Pradesh', 24: 'Gujarat', 25: 'Dadra & Nagar Haveli', 27: 'Maharashtra', 28: 'Andhra Pradesh', 29: 'Karnataka', 30: 'Goa', 31: 'Lakshadweep (U.T.)', 32: 'Kerala', 33: 'Tamilnadu', 34: 'Puducherry (U.T.)', 35: 'A and N Islands (U.T.)', 36: 'Telangana', 37: 'Ladakh (U.T.)'})
df['source_cooking'] = df['source_cooking'].replace({1: "firewood and chips", 2: "LPG", 3: "other natural gas", 4: "dung cake", 5: "kerosene", 6: "coke, coal", 7: "gobar gas", 8: "other biogas", 9: "others", 10: "charcoal", 11: "electricity", 12: "no cooking arrangement"})
df['received_subsidy_lpg'] = df['received_subsidy_lpg'].replace({1: 'yes', 2: 'no'})

# Analysis

## Primary cooking source

In [50]:
# Across India

## Calculating
trial = df.groupby(['source_cooking'], as_index=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)
display(trial)

,source_cooking,popn_size,sample_size
0,LPG,1.752651e+08,168581.0
1,charcoal,2.625950e+04,117.0
2,"coke, coal",8.845852e+05,978.0
3,dung cake,4.354240e+06,3179.0
4,electricity,2.996836e+05,344.0
5,firewood and chips,9.507038e+07,77802.0
6,gobar gas,1.494054e+05,107.0
7,kerosene,1.066841e+05,103.0
8,no cooking arrangement,8.773421e+06,5180.0
9,other biogas,9.156570e+03,8.0


In [51]:
# Primary cooking source (Rural vs Urban)

## Calculating
trial = df.groupby(['sector','source_cooking']).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)
trial = trial.reset_index()

## Pivoting
trial = trial.pivot_table(values = 'popn_size', index = ['sector'], columns = 'source_cooking')
display(trial)

source_cooking,LPG,charcoal,"coke, coal",dung cake,electricity,firewood and chips,gobar gas,kerosene,no cooking arrangement,other biogas,other natural gas,others
sector,,,,,,,,,,,,
rural,93327942.94,8082.87,515222.95,4196454.51,138373.02,90449967.65,145158.30,10495.84,2108254.86,6982.48,205417.26,2671490.57
urban,81937108.02,18176.63,369362.26,157785.26,161310.56,4620413.39,4247.07,96188.27,6665166.33,2174.09,3561192.13,294110.59


In [52]:
# Primary cooking source (Rural vs Urban by decile)

## Calculating
trial = df.groupby(['sector','decile','source_cooking'], as_index=False, observed=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)


## Pivoting
trial = trial.pivot_table (values = 'popn_size', index = ['sector', 'source_cooking'], columns = 'decile', observed = False)
display(trial)

decile                                  1            2            3   \
sector source_cooking                                                  
rural  LPG                      2189765.78   3828172.08   5028726.58   
       charcoal                        NaN          NaN          NaN   
       coke, coal                 90426.03     91748.22     70877.50   
       dung cake                 898728.58    634518.90    490343.12   
       electricity                  209.83      3245.50      5802.49   
       firewood and chips      10870772.95  10838787.05  10466555.12   
       gobar gas                   1345.60      4783.10      4993.60   
       kerosene                        NaN          NaN          NaN   
       no cooking arrangement    864697.43     82095.14     46013.55   
       other biogas                    NaN          NaN          NaN   
       other natural gas               NaN      1823.73      2612.99   
       others                    192193.91    269387.94    271466.97   
urban  LPG                      4741037.43   6249021.33   7043254.91   
       charcoal                    8781.72       701.20      2075.55   
       coke, coal                200510.66     73516.38     37281.70   
       dung cake                  60800.05     26485.26     25948.18   
       electricity                 7687.43      1706.69      8541.25   
       firewood and chips       1395107.57    884130.02    632434.81   
       gobar gas                       NaN          NaN          NaN   
       kerosene                    8348.25      6795.53     13062.37   
       no cooking arrangement    431784.02    111725.51    120816.74   
       other biogas                    NaN          NaN          NaN   
       other natural gas          16526.07     44949.63     77915.80   
       others                     49107.99     21066.23     13470.99   

decile                                  4            5           6   \
sector source_cooking                                                 
rural  LPG                      6219698.07   7151531.07  8826065.03   
       charcoal                        NaN          NaN     1302.49   
       coke, coal                 71535.78     49133.07    42611.01   
       dung cake                 454007.64    369850.00   384076.89   
       electricity                 5042.20      3188.69     5469.29   
       firewood and chips      10315812.37  10054736.79  9371727.69   
       gobar gas                   8361.80     14919.59    24380.26   
       kerosene                        NaN          NaN         NaN   
       no cooking arrangement     51174.04     36971.31    42277.05   
       other biogas                    NaN          NaN         NaN   
       other natural gas           1839.74     11127.73     7740.16   
       others                    290668.29    349336.26   317111.28   
urban  LPG                      7563867.03   8031994.27  8528772.80   
       charcoal                     820.81      2228.49      838.94   
       coke, coal                 30265.94     12090.48     4877.47   
       dung cake                  16047.30     10856.03     7463.60   
       electricity                 7178.72      8214.41    15870.00   
       firewood and chips        489825.89    350045.27   246771.13   
       gobar gas                       NaN      3002.51     1182.80   
       kerosene                   11194.41     17958.48    13603.76   
       no cooking arrangement    237119.21    229188.89   304803.21   
       other biogas                    NaN          NaN         NaN   
       other natural gas         153760.25    204358.21   241100.77   
       others                     21912.55     18734.42    30652.27   

decile                                  7            8            9   \
sector source_cooking                                                  
rural  LPG                     10739811.66  12809591.86  15706122.18   
       charcoal                     908.59      3434.29      121

## LPG subsidy - Last three months

In [53]:
# Across India
trial = df.groupby(['received_subsidy_lpg'], as_index=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)
display(trial)

,received_subsidy_lpg,popn_size,sample_size
0,no,2.047411e+08,182643.0
1,yes,8.693001e+07,79310.0


In [54]:
# LPG subsidy (Rural vs Urban)
trial = df.groupby(['sector','received_subsidy_lpg'], as_index=False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)

# Pivoting 
trial = trial.pivot_table(values = 'popn_size', index = 'received_subsidy_lpg', columns = 'sector')
display(trial)

sector,rural,urban
received_subsidy_lpg,,
no,1.339606e+08,70780475.49
yes,5.982325e+07,27106759.11


In [55]:
# LPG subsidy (Rural vs Urban by decile)
trial = df.groupby(['sector','decile','received_subsidy_lpg'], as_index=False, observed=False).apply(lambda table: pd.Series(
    {'wt_sum': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)

# Pivoting
trial = trial.pivot_table(values = 'wt_sum', index = ['sector','received_subsidy_lpg'], columns = 'decile', observed = False)
display(trial)

decile                                1            2            3   \
sector received_subsidy_lpg                                          
rural  no                    13135768.90  12385611.69  12156714.56   
       yes                    1972371.21   3368949.97   4230677.36   
urban  no                     4593144.20   4781740.27   5129570.92   
       yes                    2326546.99   2638357.51   2845231.38   

decile                                4            5            6   \
sector received_subsidy_lpg                                          
rural  no                    12371412.10  12400514.47  12621816.85   
       yes                    5046727.83   5640280.04   6400944.30   
urban  no                     5707477.17   6160735.63   6469676.06   
       yes                    2824514.94   2727935.83   2926260.69   

decile                                7            8            9   \
sector received_subsidy_lpg                                          
rural  no                    13117519.61  13675031.94  14367288.77   
       yes                    7085626.74   7717991.07   8649423.31   
urban  no                     7012139.88   7709212.94   9077767.39   
       yes                    2938841.31   2926048.03   2742495.73   

decile                                10  
sector received_subsidy_lpg               
rural  no                    17728913.76  
       yes                    9710258.77  
urban  no                    14139011.03  
       yes                    2210526.70

## LPG susbsidised cylinder - Last 3 months

In [56]:
# Across India

## Filtering for people who recieved subsidy
trial = df.copy()
trial = trial.loc[trial['received_subsidy_lpg'] == 'yes']

# Calculation
trial = trial.groupby('num_subsidized_lpg', as_index = False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)
display(trial)

,num_subsidized_lpg,popn_size,sample_size
0,1.0,50561433.32,44578.0
1,2.0,26125750.02,24290.0
2,3.0,10242826.37,10442.0


In [57]:
# Rural vs Urban

## Filtering for people who recieved subsidy
trial = df.copy()
trial = trial.loc[trial['received_subsidy_lpg'] == 'yes']

# Calculation
trial = trial.groupby(['sector','num_subsidized_lpg']).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)
display(trial)

popn_size  sample_size
sector num_subsidized_lpg                          
rural  1.0                 37768724.69      28640.0
       2.0                 16147548.68      12330.0
       3.0                  5906977.23       4628.0
urban  1.0                 12792708.63      15938.0
       2.0                  9978201.34      11960.0
       3.0                  4335849.14       5814.0

In [58]:
# Rural vs Urban across deciles

## Filtering for people who recieved subsidy
trial = df.copy()
trial = trial.loc[trial['received_subsidy_lpg'] == 'yes']

## Calculation
trial = trial.groupby(['sector', 'decile', 'num_subsidized_lpg'], observed = False).apply(lambda table: pd.Series(
    {'popn_size': table['multiplier'].sum()/100,
     'sample_size': table.shape[0]}
), include_groups = False)
trial = trial.reset_index()

# Pivotting
trial = trial.pivot_table(values = 'popn_size', index = ['sector', 'num_subsidized_lpg'], columns = 'decile',observed = False)
display(trial)

decile                             1           2           3           4   \
sector num_subsidized_lpg                                                   
rural  1.0                 1401071.76  2307225.55  2748154.71  3195400.48   
       2.0                  372198.76   719902.11  1026512.97  1284279.57   
       3.0                  199100.69   341822.31   456009.68   567047.78   
urban  1.0                 1082303.94  1145015.17  1280678.40  1310443.93   
       2.0                  763540.36  1004242.29  1055779.08  1042076.01   
       3.0                  480702.69   489100.05   508773.90   471995.00   

decile                             5           6           7           8   \
sector num_subsidized_lpg                                                   
rural  1.0                 3622143.60  4006361.55  4451812.55  4800677.95   
       2.0                 1437827.10  1785563.03  1958060.45  2207975.32   
       3.0                  580309.34   609019.72   675753.74   709337.80   
urban  1.0                 1296553.52  1346444.61  1450444.23  1431786.78   
       2.0                 1015353.85  1146819.64  1057620.12  1128751.96   
       3.0                  416028.46   432996.44   430776.96   365509.29   

decile                             9           10  
sector num_subsidized_lpg                          
rural  1.0                 5352931.92  5882944.62  
       2.0                 2521832.85  2833396.52  
       3.0                  774658.54   993917.63  
urban  1.0                 1385634.43  1063403.62  
       2.0                  946953.98   817064.05  
       3.0                  409907.32   330059.03

## Average quantity and value of LPG and firewood consumed

* Values are reported for past 30 days

In [10]:
# Across India

## Duplicate df
trial = df.copy()

## Sumproduct of consumption
value_mult = pd.Series({
    'lpg_qty': (trial['lpg_qty'] * trial['multiplier']).sum(),
    'lpg_value':(trial['lpg_value'] * trial['multiplier']).sum(),
    'firewood_qty': (trial['firewood_qty'] * trial['multiplier']).sum(),
    'firewood_value':(trial['firewood_value'] * trial['multiplier']).sum()
}).round(2)

## Sumproduct of hh size
hh_size_mult = (trial['hh_size']*trial['multiplier']).sum()

# Weighted averages
trial = value_mult/hh_size_mult
display(trial)

lpg_qty             1.598124
lpg_value         106.070636
firewood_qty        9.937351
firewood_value     46.950929
dtype: float64

In [11]:
# Rural vs Urban

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby('sector').apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum(),
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewod_qty': (table['firewood_qty']*table['multiplier']).sum(),
    'wt_firewood_value': (table['firewood_value']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby('sector').apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

,wt_lpg_qty,wt_lpg_value,wt_firewod_qty,wt_firewood_value
sector,,,,
rural,1.270472,84.779279,13.300155,62.60823
urban,2.369823,156.216889,2.017142,10.07423


In [52]:
# Rural vs Urban across deciles

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'decile'], observed=False).apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum(),
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewod_qty': (table['firewood_qty']*table['multiplier']).sum(),
    'wt_firewood_value': (table['firewood_value']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'decile'], observed=False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0).round(2)
display(trial)

wt_lpg_qty  wt_lpg_value  wt_firewod_qty  wt_firewood_value
sector decile                                                             
rural  1             0.39         26.79           15.03              66.63
       2             0.66         44.75           15.22              69.26
       3             0.83         55.93           14.96              69.12
       4             0.99         66.54           14.43              68.61
       5             1.10         73.44           13.90              66.80
       6             1.27         84.82           13.48              64.89
       7             1.46         97.54           12.59              60.64
       8             1.66        110.09           12.01              58.06
       9             1.92        127.53           11.07              53.08
       10            2.41        160.36           10.31              48.98
urban  1             1.58        104.62            4.88              24.00
       2             1.99        131.35            3.34              17.37
       3             2.18        143.11            2.62              13.23
       4             2.27        149.70            2.12              10.36
       5             2.38        156.70            1.91               9.31
       6             2.49        163.90            1.44               7.25
       7             2.61        171.94            1.35               6.70
       8             2.67        175.65            1.12               5.35
       9             2.79        183.74            0.79               3.98
       10            2.74        181.46            0.61               3.20

In [12]:
# Rural vs Urban across states

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty'] * table['multiplier']).sum(),
    'wt_lpg_value':(table['lpg_value'] * table['multiplier']).sum(),
    'wt_firewood_qty': (table['firewood_qty'] * table['multiplier']).sum(),
    'wt_firewood_value': (table['firewood_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

wt_lpg_qty  wt_lpg_value  wt_firewood_qty  \
sector state                                                               
rural  A and N Islands (U.T.)    2.935685    193.343921         1.297966   
       Andhra Pradesh            1.881545    127.873048         6.425040   
       Arunachal Pradesh         1.257256     88.357207        18.783891   
       Assam                     0.990500     66.470068        21.029068   
       Bihar                     1.095967     77.998667         7.409549   
...                                   ...           ...              ...   
urban  Telangana                 2.047571    138.812165         0.123514   
       Tripura                   2.366207    179.032254        10.530072   
       Uttar Pradesh             2.279728    147.500488         1.162149   
       Uttrakhand                2.800773    178.980782         1.175975   
       West Bengal               2.789188    180.640258         3.241481   

                               wt_firewood_value  
sector state                                      
rural  A and N Islands (U.T.)           6.060627  
       Andhra Pradesh                  26.289377  
       Arunachal Pradesh              121.369622  
       Assam                           98.147684  
       Bihar                           53.221343  
...                                          ...  
urban  Telangana                        0.860567  
       Tripura                         55.055257  
       Uttar Pradesh                    7.105410  
       Uttrakhand                       5.868351  
       West Bengal                     16.999104  

[72 rows x 4 columns]

In [15]:
trial[trial.index.get_level_values(0) == 'urban']

wt_lpg_qty  wt_lpg_value  wt_firewood_qty  \
sector state                                                               
urban  A and N Islands (U.T.)    3.446384    235.749064         0.214033   
       Andhra Pradesh            2.180879    147.770615         0.758318   
       Arunachal Pradesh         2.411137    173.941360        10.266982   
       Assam                     2.604416    177.177031         2.012344   
       Bihar                     2.236689    160.099878         2.053898   
       Chandigarh(U.T.)          2.925051    201.796932         0.026843   
       Chattisgarh               2.318764    158.815367         2.579977   
       Dadra & Nagar Haveli      2.523876    163.694446         2.506612   
       Delhi                     2.249330    145.565912         0.021515   
       Goa                       3.187415    198.297461         0.736371   
       Gujarat                   1.569232    103.851594         0.788378   
       Haryana                   2.855434    180.579407         0.613236   
       Himachal Pradesh          3.018666    214.351777         3.354114   
       Jammu & Kashmir           2.353751    166.895077         1.615918   
       Jharkhand                 2.084631    144.911570         2.476957   
       Karnataka                 2.400978    156.425165         2.384925   
       Kerala                    2.439940    159.874414        12.124831   
       Ladakh (U.T.)             2.362640    180.447101         0.116185   
       Lakshadweep (U.T.)        1.584744    157.576844         5.363044   
       Madhya Pradesh            2.129490    142.023456         1.296701   
       Maharashtra               2.464070    158.547105         0.541587   
       Manipur                   1.996565    158.610834         0.546643   
       Meghalaya                 2.286423    170.392722         3.738069   
       Mizoram                   3.528328    239.288750         0.068226   
       Nagaland                  2.178764    173.176309        19.769421   
       Odisha                    2.243735    148.004000         9.206691   
       Puducherry (U.T.)         2.929160    201.693910         0.118627   
       Punjab                    3.107593    202.303450         0.538215   
       Rajasthan                 2.281580    146.731027         3.266773   
       Sikkim                    2.743746    248.896237         0.284645   
       Tamilnadu                 2.634196    178.476598         0.709069   
       Telangana                 2.047571    138.812165         0.123514   
       Tripura                   2.366207    179.032254        10.530072   
       Uttar Pradesh             2.279728    147.500488         1.162149   
       Uttrakhand                2.800773    178.980782         1.175975   
       West Bengal               2.789188    180.640258         3.241481   

                               wt_firewood_value  
sector state                                      
urban  A and N Islands (U.T.)           0.853284  
       Andhra Pradesh                   3.808627  
       Arunachal Pradesh               67.898802  
       Assam                            9.431749  
       Bihar                           14.849673  
       Chandigarh(U.T.)                 0.214747  
       Chattisgarh                     14.502700  
       Dadra & Nagar Haveli            12.889267  
       Delhi                            0.193664  
       Goa                              2.944782  
       Gujarat                          4.256246  
       Haryana                          4.223894  
       Himachal Pradesh                11.887311  
       Jammu & Kashmir                  6.222001  
       Jharkhand                       10.888987  
       Karnataka                       12.267884  
       Kerala                          48.972498  
       Ladakh (U.T.)                    1.036975  
       Lakshadweep (U.T.)              28.451371  
       Madhya Pradesh                   5.743507  
       Maharashtra  

## Average quantity and value of firewood consumed from market

In [110]:
# Across India

## Duplicate df
trial = df.copy()

## Sumproduct of consumption
value_mult = pd.Series({
    'firewood_mkt_qty': (trial['firewood_mkt_qty'] * trial['multiplier']).sum(),
    'firewood_mkt_value':(trial['firewood_mkt_value'] * trial['multiplier']).sum()
}).round(2)

## Sumproduct of hh size
hh_size_mult = (trial['hh_size']*trial['multiplier']).sum()

# Weighted averages
trial = value_mult/hh_size_mult
display(trial)

firewood_mkt_qty       6.965658
firewood_mkt_value    33.798832
dtype: float64

In [125]:
# Rural vs Urban

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby('sector').apply(lambda table: pd.Series({
    'wt_firewood_mkt_qty': (table['firewood_mkt_qty'] * table['multiplier']).sum(),
    'wt_firewood_mkt_value':(table['firewood_mkt_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby('sector').apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

,wt_firewood_mkt_qty,wt_firewood_mkt_value
sector,,
rural,9.254475,44.566351
urban,1.574948,8.438745


In [127]:
# Rural vs Urban across deciles

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'decile']).apply(lambda table: pd.Series({
    'wt_firewood_mkt_qty': (table['firewood_mkt_qty'] * table['multiplier']).sum(),
    'wt_firewood_mkt_value':(table['firewood_mkt_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'decile']).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

wt_firewood_mkt_qty  wt_firewood_mkt_value
sector decile                                            
rural  1                 12.304579              54.413062
       2                 11.530213              53.543743
       3                 10.885525              51.120064
       4                 10.227797              49.686170
       5                  9.593851              47.209538
       6                  8.857644              43.942270
       7                  8.304052              41.296977
       8                  7.827312              39.188145
       9                  7.015588              35.049399
       10                 5.997914              30.213107
urban  1                  4.331902              21.566056
       2                  2.848006              15.340643
       3                  2.072258              11.267240
       4                  1.646409               8.712177
       5                  1.420490               7.586944
       6                  0.987593               5.683504
       7                  0.869814               5.074928
       8                  0.696843               3.911090
       9                  0.510051               2.984900
       10                 0.365950               2.259185

In [27]:
# Rural vs Urban across states

# Duplicate df and replace Nan values
trial = df.copy()


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: pd.Series({
    'wt_firewood_mkt_qty': (table['firewood_mkt_qty'] * table['multiplier']).sum(),
    'wt_firewood_mkt_value':(table['firewood_mkt_value'] * table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector', 'state'], observed = False).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

trial = value_mult.div(hh_size_mult, axis = 0)
display(trial)

wt_firewood_mkt_qty  wt_firewood_mkt_value
sector state                                                             
rural  A and N Islands (U.T.)             1.297966               6.060627
       Andhra Pradesh                     6.337395              26.043898
       Arunachal Pradesh                 16.854789             111.634300
       Assam                             12.851265              65.990285
       Bihar                              5.368933              38.197365
...                                            ...                    ...
urban  Telangana                          0.123115               0.857575
       Tripura                            8.572852              46.564274
       Uttar Pradesh                      1.002565               6.393113
       Uttrakhand                         1.169996               5.838457
       West Bengal                        2.846219              15.128429

[72 rows x 2 columns]

In [28]:
trial[trial.index.get_level_values(0) == 'urban']

wt_firewood_mkt_qty  wt_firewood_mkt_value
sector state                                                             
urban  A and N Islands (U.T.)             0.214033               0.853284
       Andhra Pradesh                     0.755151               3.791601
       Arunachal Pradesh                  9.150919              60.658396
       Assam                              1.506175               7.378310
       Bihar                              1.645396              11.821243
       Chandigarh(U.T.)                   0.026843               0.214747
       Chattisgarh                        2.280275              12.776171
       Dadra & Nagar Haveli               2.314554              11.806258
       Delhi                              0.021515               0.193664
       Goa                                0.642300               2.568494
       Gujarat                            0.749301               3.966844
       Haryana                            0.596889               4.142647
       Himachal Pradesh                   2.037785               6.597161
       Jammu & Kashmir                    1.329779               5.186638
       Jharkhand                          2.249808               9.812565
       Karnataka                          2.102091              10.894142
       Kerala                             6.372496              31.105580
       Ladakh (U.T.)                      0.116185               1.036975
       Lakshadweep (U.T.)                 3.247482              18.367614
       Madhya Pradesh                     1.075819               4.794654
       Maharashtra                        0.502864               2.584978
       Manipur                            0.489822               4.664711
       Meghalaya                          2.963309              15.896506
       Mizoram                            0.000000               0.000000
       Nagaland                           9.326515              37.503127
       Odisha                             8.872845              42.164875
       Puducherry (U.T.)                  0.118627               1.372400
       Punjab                             0.526855               4.339808
       Rajasthan                          2.707494              14.344861
       Sikkim                             0.284645               0.755628
       Tamilnadu                          0.685830               5.276920
       Telangana                          0.123115               0.857575
       Tripura                            8.572852              46.564274
       Uttar Pradesh                      1.002565               6.393113
       Uttrakhand                         1.169996               5.838457
       West Bengal                        2.846219              15.128429

## Household average annual cylinder consumption

In [19]:
# Across India

## Duplicate df
trial = df.copy()

## Sumproduct of consumption
value_mult = pd.Series({
    'lpg_qty': (trial['lpg_qty'] * trial['multiplier']).sum()/trial['multiplier'].sum(),
})

#Converting to annual cylinder consumption
value_mult = round(value_mult*(12/14.2),2)

display(value_mult)

lpg_qty    5.64
dtype: float64

In [21]:
# Rural vs Urban

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby('sector').apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum()/table['multiplier'].sum(),
    }), include_groups = False)

#Converting to annual cylinder consumption
value_mult = round(value_mult*(12/14.2),2)

display(value_mult)

,wt_lpg_qty
sector,
rural,4.74
urban,7.42


In [22]:
# Rural vs Urban across deciles

# Duplicate df and replace Nan values
trial = df.copy()
trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']] = trial[['lpg_qty', 'lpg_value', 'firewood_qty', 'firewood_value']].fillna(0)


# Sumproduct of consumption
value_mult = trial.groupby(['sector', 'decile'], observed=False).apply(lambda table: pd.Series({
    'wt_lpg_qty': (table['lpg_qty']*table['multiplier']).sum()/table['multiplier'].sum(),
    }), include_groups = False).round(2)

#Converting to annual cylinder consumption
value_mult = round(value_mult*(12/14.2),2)

display(value_mult)

wt_lpg_qty
sector decile            
rural  1             1.88
       2             3.04
       3             3.67
       4             4.12
       5             4.39
       6             4.83
       7             5.23
       8             5.59
       9             6.04
       10            6.35
urban  1             7.01
       2             8.22
       3             8.37
       4             8.17
       5             8.21
       6             8.13
       7             8.03
       8             7.70
       9             7.24
       10            5.15

## Share of HH expenditure on LPG

In [34]:
# Across India

## Duplicate df
trial = df.copy()

## Filtering for HHs with primary cooking source as firewood or LPG
trial = trial.loc[trial['source_cooking'].isin(['LPG','firewood and chips']),:]

## Replace NA values with 0
trial[['lpg_value', 'firewood_mkt_value']] = trial[['lpg_value', 'firewood_mkt_value']].fillna(0)

## Total expenditure
trial['monthly_totalexp'] = trial['hh_size']*trial['mpce']

## Checking groupsize
group_size = trial.groupby('source_cooking').size()
group_size.name = 'sample size'
display(pd.DataFrame(group_size))


## Sumproduct of consumption (Note: LPG and firewood consumption are recorded for last month. Therefore, no scaling required)
value_mult = trial.groupby('source_cooking').apply(lambda table: pd.Series({
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewood_mkt_value': (table['firewood_mkt_value']*table['multiplier']).sum(),
    'wt_total_exp': (table['monthly_totalexp']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby('source_cooking').apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

# Average monthly per capita expenditure 
trial = value_mult.div(hh_size_mult, axis = 0)
trial = trial.rename(columns = {'wt_lpg_value': 'LPG MPCE','wt_firewood_mkt_value': 'Firewood mkt MPCE','wt_total_exp': 'Overall MPCE' })

display(trial)

,sample size
source_cooking,
LPG,168581
firewood and chips,77802


,LPG MPCE,Firewood mkt MPCE,Overall MPCE
source_cooking,,,
LPG,157.022996,10.625108,5718.144013
firewood and chips,34.421541,76.116185,3557.139368


In [35]:
# Across Rural vs Urban

## Duplicate df
trial = df.copy()

## Filtering for HHs with primary cooking source as firewood or LPG
trial = trial.loc[trial['source_cooking'].isin(['LPG','firewood and chips']),:]

## Replace NA values with 0
trial[['lpg_value', 'firewood_mkt_value']] = trial[['lpg_value', 'firewood_mkt_value']].fillna(0)

## Total expenditure
trial['monthly_totalexp'] = trial['hh_size']*trial['mpce']

## Checking groupsize
group_size = trial.groupby(['sector','source_cooking']).size()
group_size.name = 'sample size'
display(pd.DataFrame(group_size))


## Sumproduct of consumption (Note: LPG and firewood consumption are recorded for last month. Therefore, no scaling required)
value_mult = trial.groupby(['sector','source_cooking']).apply(lambda table: pd.Series({
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewood_mkt_value': (table['firewood_mkt_value']*table['multiplier']).sum(),
    'wt_total_exp': (table['monthly_totalexp']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector','source_cooking']).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

# Average monthly per capita expenditure 
trial = value_mult.div(hh_size_mult, axis = 0)
trial = trial.rename(columns = {'wt_lpg_value': 'LPG MPCE','wt_firewood_mkt_value': 'Firewood mkt MPCE','wt_total_exp': 'Overall MPCE' })

display(trial)

sample size
sector source_cooking                 
rural  LPG                       75775
       firewood and chips        72246
urban  LPG                       92806
       firewood and chips         5556

LPG MPCE  Firewood mkt MPCE  Overall MPCE
sector source_cooking                                                 
rural  LPG                 143.632662          16.071894   4794.049507
       firewood and chips   33.556451          75.242904   3525.958495
urban  LPG                 173.891778           3.763395   6882.293466
       firewood and chips   53.088018          94.959380   4229.944193

In [36]:
# Rural across deciles

## Duplicate df
trial = df.copy()

## Filtering for HHs with primary cooking source as firewood or LPG
trial = trial.loc[trial['source_cooking'].isin(['LPG','firewood and chips']),:]

## Replace NA values with 0
trial[['lpg_value', 'firewood_mkt_value']] = trial[['lpg_value', 'firewood_mkt_value']].fillna(0)

## Total expenditure
trial['monthly_totalexp'] = trial['hh_size']*trial['mpce']

## Checking groupsize
group_size = trial.groupby(['sector','source_cooking', 'decile']).size()
group_size.name = 'sample size'
display(pd.DataFrame(group_size))


## Sumproduct of consumption (Note: LPG and firewood consumption are recorded for last month. Therefore, no scaling required)
value_mult = trial.groupby(['sector','source_cooking', 'decile']).apply(lambda table: pd.Series({
    'wt_lpg_value': (table['lpg_value']*table['multiplier']).sum(),
    'wt_firewood_mkt_value': (table['firewood_mkt_value']*table['multiplier']).sum(),
    'wt_total_exp': (table['monthly_totalexp']*table['multiplier']).sum()
    }), include_groups = False).round(2)

## Sumproduct of hh size 
hh_size_mult = trial.groupby(['sector','source_cooking', 'decile']).apply(lambda table: (table['multiplier']*table['hh_size']).sum(), include_groups = False)

# Average monthly per capita expenditure 
trial = value_mult.div(hh_size_mult, axis = 0)
trial = trial.rename(columns = {'wt_lpg_value': 'LPG MPCE','wt_firewood_mkt_value': 'Firewood MPCE','wt_total_exp': 'Overall MPCE' })

display(trial)

sample size
sector source_cooking     decile             
rural  LPG                1              1660
                          2              2951
                          3              3844
                          4              4742
                          5              5547
                          6              6814
                          7              8357
                          8             10142
                          9             12651
                          10            19067
       firewood and chips 1              7977
                          2              8081
                          3              7846
                          4              7872
                          5              7752
                          6              7335
                          7              6978
                          8              6623
                          9              6058
                          10             5724
urban  LPG                1              5663
                          2              7113
                          3              8021
                          4              8483
                          5              8964
                          6              9526
                          7             10002
                          8             10650
                          9             11412
                          10            12972
       firewood and chips 1              1620
                          2               993
                          3               725
                          4               586
                          5               461
                          6               356
                          7               328
                          8               238
                          9               144
                          10              105

LPG MPCE  Firewood MPCE  Overall MPCE
sector source_cooking     decile                                         
rural  LPG                1       100.771414      12.696155   1982.231020
                          2       110.760828      13.328361   2482.350295
                          3       117.983548      13.915350   2837.327068
                          4       124.255527      15.164836   3164.625104
                          5       128.809155      15.217227   3501.755586
                          6       133.967553      15.710966   3870.788723
                          7       140.673650      15.835159   4307.452740
                          8       147.177307      17.255720   4892.005177
                          9       157.369668      17.901901   5773.594502
                          10      186.099566      17.392726   8533.131812
       firewood and chips 1        12.118584      70.203419   1896.880883
                          2        21.093559      73.652475   2470.044037
                          3        25.747469      73.899306   2830.811594
                          4        31.039792      75.638813   3160.261508
                          5        34.036114      74.699002   3494.581106
                          6        39.509023      75.312850   3861.313222
                          7        45.395283      78.019301   4299.147984
                          8        50.621357      80.150130   4875.491940
                          9        59.932914      80.938804   5735.450425
                          10       78.002784      81.063309   8384.379004
urban  LPG                1       129.722551       5.747079   2785.922311
                          2       145.328754       5.250816   3689.922817
                          3       153.627330       4.489748   4353.693512
                          4       159.838494       4.327430   4979.157549
                          5       166.381751       4.523736   5620.471796
                          6       173.502431       3.521484   6333.804254
                          7       182.779060       3.080743   7197.831687
                          8       189.315309       2.622690   8352.192861
                          9       204.892784       2.170284  10129.780501
                          10      234.309888       2.032973  16021.008159
       firewood and chips 1        34.850559      86.763270   2631.144924
                          2        43.988126      98.130481   3664.221472
                          3        55.917856      97.271001   4348.301288
                          4        68.923271      93.598974   4961.285813
                          5        73.794464      98.813704   5601.702811
                          6        84.261061     100.856721   6323.636261
                          7        87.544249     110.800211   7205.850534
                          8        97.762607     103.010855   8318.483418
                          9       100.384375     132.927637  10013.665757
                          10      120.325257     120.120489  15715.834461